In [2]:
import cv2
import numpy as np
import csv
import os


def load_csv_data(file_paths):
    all_face_positions = []
    for file_path in file_paths:
        with open(file_path, "r") as csvfile:
            csvreader = csv.reader(csvfile)
            next(csvreader)  # Skip header
            for row in csvreader:
                frame, x, y, w, h, eye_point1, eye_point2 = row
                all_face_positions.append((int(x), int(y), int(w), int(h)))
    return all_face_positions


def draw_face_centers_on_white_background(face_positions, image_size=(1080, 1920)):
    # Create a white background image
    white_background = np.ones((image_size[0], image_size[1], 3), dtype=np.uint8) * 255

    # Define lines based on the example dimensions
    width, height = image_size[1], image_size[0]
    line1_x = int(width * 0.35)
    line2_x = int(width * 0.65)
    line1_y = int(height * 0.25)
    line2_y = int(height * 0.6)

    # Draw lines (blue color #4BEEEE)
    line_color = (235, 64, 52)
    cv2.line(white_background, (line1_x, 0), (line1_x, height), line_color, 2)
    cv2.line(white_background, (line2_x, 0), (line2_x, height), line_color, 2)
    cv2.line(white_background, (0, line1_y), (width, line1_y), line_color, 2)
    cv2.line(white_background, (0, line2_y), (width, line2_y), line_color, 2)

    # Draw face centers (red color #EE4B4B)
    circle_color = (66, 135, 245)
    for x, y, w, h in face_positions:
        if w > 0 and h > 0:  # Only draw for valid positions
            center_x = x + w // 2
            center_y = y + h // 2
            cv2.circle(
                white_background, (center_x, center_y), 5, circle_color, -1
            )  # Draw red circle

    output_image_path = "output_with_face_centers_and_lines.jpg"
    cv2.imwrite(output_image_path, white_background)
    print(f"Image saved to {output_image_path}")


# Example usage
csv_files = [
    "output_ive_baddie_1.csv",
    "output_ive_baddie_2.csv",
    "output_ive_baddie_3.csv",
    "output_ive_baddie_4.csv",
    "output_ive_baddie_5.csv",
    "output_ive_baddie_6.csv",
]  # Replace with your CSV file paths

face_positions = load_csv_data(csv_files)
draw_face_centers_on_white_background(face_positions)

Image saved to output_with_face_centers_and_lines.jpg


In [1]:
import cv2
import numpy as np
import os
from insightface.app import FaceAnalysis
import csv
import time
from multiprocessing import Pool, cpu_count
from tqdm.notebook import tqdm


def process_video(video_path):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"No file found at {video_path}")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception("Failed to open video file.")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps if fps else 0

    app = FaceAnalysis(
        allowed_modules=["detection", "landmark_2d_106"],
        providers=["CUDAExecutionProvider"],
    )
    app.prepare(ctx_id=0, det_size=(640, 640))

    frame_count = 0
    face_positions = []

    with tqdm(total=total_frames // 5, desc=os.path.basename(video_path)) as pbar:
        while frame_count < total_frames:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            if frame_count % 5 != 0:
                continue

            faces = app.get(frame)
            current_frame_positions = []

            for face in faces:
                bbox = face["bbox"].astype(int)
                current_frame_positions.append(
                    (bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1])
                )

            if current_frame_positions:
                face_positions.append(current_frame_positions)
            else:
                face_positions.append([(0, 0, 0, 0)])

            pbar.update(1)

    cap.release()
    return face_positions, fps, total_frames, duration


def save_face_positions_to_csv(face_positions, csv_path):
    with open(csv_path, "w", newline="") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["frame", "x", "y", "w", "h"])
        for i, positions in enumerate(face_positions):
            for position in positions:
                x, y, w, h = position
                csvwriter.writerow([i * 5, x, y, w, h])


def load_csv_data(file_paths):
    all_face_positions = []
    for file_path in file_paths:
        with open(file_path, "r") as csvfile:
            csvreader = csv.reader(csvfile)
            next(csvreader)  # Skip header
            for row in csvreader:
                frame, x, y, w, h = row[:5]
                all_face_positions.append((int(x), int(y), int(w), int(h)))
    return all_face_positions


def draw_face_centers_on_white_background(face_positions, image_size=(1080, 1920)):
    # Create a white background image
    white_background = np.ones((image_size[0], image_size[1], 3), dtype=np.uint8) * 255

    # Define lines based on the example dimensions
    width, height = image_size[1], image_size[0]
    line1_x = int(width * 0.35)
    line2_x = int(width * 0.65)
    line1_y = int(height * 0.25)
    line2_y = int(height * 0.6)

    # Draw lines (blue color #4BEEEE)
    line_color = (75, 238, 238)
    cv2.line(white_background, (line1_x, 0), (line1_x, height), line_color, 2)
    cv2.line(white_background, (line2_x, 0), (line2_x, height), line_color, 2)
    cv2.line(white_background, (0, line1_y), (width, line1_y), line_color, 2)
    cv2.line(white_background, (0, line2_y), (width, line2_y), line_color, 2)

    # Draw face centers (red color #EE4B4B)
    circle_color = (238, 75, 75)
    for x, y, w, h in face_positions:
        if w > 0 and h > 0:  # Only draw for valid positions
            center_x = x + w // 2
            center_y = y + h // 2
            cv2.circle(
                white_background, (center_x, center_y), 5, circle_color, -1
            )  # Draw red circle

    output_image_path = "output_with_face_centers_and_lines_on_white.jpg"
    cv2.imwrite(output_image_path, white_background)
    print(f"Image saved to {output_image_path}")


def process_video_multiprocessing(video_paths):
    with Pool(cpu_count()) as pool:
        results = pool.map(process_video, video_paths)
    return results


if __name__ == "__main__":
    start_time = time.time()

    folder_path = "./data/"  # Folder path for storing videos
    video_files = [
        "ive_baddie_1.mp4",
        "ive_baddie_2.mp4",
        "ive_baddie_3.mp4",
        "ive_baddie_4.mp4",
        "ive_baddie_5.mp4",
        "ive_baddie_6.mp4",
    ]
    video_paths = [os.path.join(folder_path, video_file) for video_file in video_files]
    results = process_video_multiprocessing(video_paths)

    # Save results to CSV files
    csv_files = []
    for i, result in enumerate(results):
        face_positions, fps, total_frames, duration = result
        csv_path = f"output_{i + 1}.csv"
        save_face_positions_to_csv(face_positions, csv_path)
        csv_files.append(csv_path)

    face_positions = load_csv_data(csv_files)
    draw_face_centers_on_white_background(face_positions)

    finished_time = time.time()
    processed_time = finished_time - start_time
    print("------------------------------------------------------------")
    print(f"Processed time: {processed_time:.2f} seconds")

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.10 (you have 1.4.9). Upgrade using: pip install --upgrade albumentations
